Paper Link
https://www.mdpi.com/2306-5729/3/2/15/htm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import string
torch.manual_seed(1)

In [5]:
file = open('/content/drive/My Drive/DSi/Data Science/DATA/bangla_sentiment.tsv', encoding ='utf-8')

data = list(file)
#print(data[1])
X = []
for i in range(1, len(data)):
    data_split = data[i].split('\t')
    #print(data_split[1])
    X.append(data_split[1])

word_to_ix = {}
ix_to_word = {}    
 
inputs = []    
for i in range(len(X)):
    sentence = X[i].translate(str.maketrans('', '', string.punctuation))
    #print(sentence)
    #x = sentence.split()
    for word in sentence.split():
        inputs.append(word)
#print(inputs)

for i, word in enumerate(inputs):
    #print(i, word)
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)
        ix_to_word[len(ix_to_word)] = word
#print(word_to_ix['খুব'], ix_to_word[28])
final_data = []
for i in range(2, len(inputs) - 2):
    context = [ inputs[i - 2], inputs[i -1], inputs[i + 1], inputs[i + 2] ]
    target = [ inputs[i] ]
    
    final_data.append((context, target))
print(final_data[1])



(['কিন্তু', 'আমাদের', 'ভয়ঙ্কর', 'ছিল।'], ['জন্য'])


In [6]:
class bangla_word_embedding(nn.Module):
    
    def __init__(self, VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM):
        
        super(bangla_word_embedding, self).__init__()
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.linear1 = nn.Linear(EMBEDDING_DIM*4, HIDDEN_DIM)
        self.linear2 = nn.Linear(HIDDEN_DIM, VOCAB_SIZE)
        
    def forward(self, inputs):
        
        #print('ENTERED FORWARD = {}'.format(list(inputs.shape)))
        outputs = self.embedding(inputs)
        #print('Returned from Embedding = {}'.format( list(outputs.shape) ))
        outputs = outputs.view(1, -1)
        #print('reshaping to 4 row to 1 = {}'.format( list(outputs.shape) ))
        outputs = self.linear1(outputs)
        #print('Returned from LINEAR1 = {}'.format( list(outputs.shape)  ))
        outputs = self.linear2(outputs)
        #print('Output Returned from LINEAR2 = {}'.format( list(outputs.shape) ))
        
        return outputs
        

In [7]:
VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)
EMBEDDING_DIM = 64
HIDDEN_DIM = 128

model = bangla_word_embedding(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)
weight_decay=10
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay=1000)
#optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_function = nn.NLLLoss()

4863


In [8]:
def make_tensor(inputs, dictionary):
    x = torch.tensor( [ word_to_ix[word] for word in inputs ], dtype=torch.long )
    return x

In [ ]:
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

0
1
Tesla P100-PCIE-16GB
True


device(type='cuda')

In [ ]:
model.to(device)

bangla_word_embedding(
  (embedding): Embedding(4863, 64)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=4863, bias=True)
)

In [ ]:
EPOCH = 6
grad_clip = 2
for i in range(EPOCH):
    total_loss = 0;
    for X, Y in final_data:
        #print(X, Y)

        X = make_tensor(X, word_to_ix).to(device)
        Y = make_tensor(Y, word_to_ix).to(device)
        #print('X shape = {}, Y shape = {}'.format(X.shape, Y.shape))

        model.zero_grad()

        outputs = model(X)
        #print('returned outputs size = {}'.format( list(outputs.shape) ))
        loss = loss_function(outputs, Y)
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(),grad_clip)
        optimizer.step()
        total_loss += loss.item()
        
    print('epoch = {}, loss = {}'.format(i, total_loss))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


epoch = 0, loss = 0.036504485902318036
epoch = 1, loss = 0.03559420504125473
epoch = 2, loss = 0.03559420504125473
epoch = 3, loss = 0.03559420504125473
epoch = 4, loss = 0.03559420504125473
epoch = 5, loss = 0.03559420504125473


### SAVE

In [ ]:
#model.state_dict()
torch.save(model.state_dict(), '/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW.pth')

### LOAD

In [9]:

VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)
EMBEDDING_DIM = 64
HIDDEN_DIM = 128

model = bangla_word_embedding(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)
model.load_state_dict(torch.load('/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW.pth'))
model.eval()

4863


bangla_word_embedding(
  (embedding): Embedding(4863, 64)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=4863, bias=True)
)

In [ ]:
model = model.to('cpu')
model = model.eval()
with open('/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW.tsv', 'w') as f:
  with torch.no_grad():
    for i, word in enumerate(word_to_ix):
      #print(i, word)
      #print(ix_to_word[i])
      v = model.embedding( torch.tensor(word_to_ix[word], dtype=torch.long ))
      print('size = {}, values = {}'.format(v.size(),v))
      #f.write('{}\t'.format(ix_to_word[i]))
      for value in v:  
        f.write('{}\t'.format(value))
      f.write('\n')

      # if i == 10:
      #   break;
      #Vector = [  ]

with open('/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW_meta.tsv', 'w') as f:
  with torch.no_grad():
    for i, word in enumerate(word_to_ix):
      #print(i, word)
      #print(ix_to_word[i])
      #v = model.embedding( torch.tensor(word_to_ix[word], dtype=torch.long ))
      #print('size = {}, values = {}'.format(v.size(),v))
      #f.write('{}\t'.format(ix_to_word[i]))  
      f.write('{}\n'.format(ix_to_word[i]))

Streaming output truncated to the last 5000 lines.
        -2.8026e-45,  0.0000e+00,  1.4013e-45, -2.8026e-45,  2.8026e-45,
        -1.4013e-45, -2.8026e-45, -1.4013e-45,  1.4013e-45,  4.2039e-45,
        -2.8026e-45, -1.4013e-45, -1.4013e-45, -0.0000e+00, -1.4013e-45,
        -0.0000e+00,  2.8026e-45,  0.0000e+00, -0.0000e+00,  2.8026e-45,
        -0.0000e+00,  1.4013e-45,  1.4013e-45,  0.0000e+00, -1.4013e-45,
         1.4013e-45,  0.0000e+00,  1.4013e-45, -1.4013e-45,  1.4013e-45,
         1.4013e-45,  1.4013e-45,  1.4013e-45,  0.0000e+00])
size = torch.Size([64]), values = tensor([ 1.4013e-45, -0.0000e+00, -1.4013e-45, -2.8026e-45, -0.0000e+00,
        -0.0000e+00, -1.4013e-45, -2.8026e-45,  0.0000e+00, -2.8026e-45,
        -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  1.4013e-45,
         1.4013e-45,  1.4013e-45,  2.8026e-45,  2.8026e-45, -2.8026e-45,
         0.0000e+00,  1.4013e-45, -1.4013e-45, -1.4013e-45, -1.4013e-45,
        -1.4013e-45,  1.4013e-45,  0.0000e+00, -4.2

# JUST TEST

In [10]:
model = model.to('cpu')
model = model.eval()
with open('/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW1.tsv', 'w') as f:
  with torch.no_grad():
    for i, word in enumerate(word_to_ix):
      #print(i, word)
      #print(ix_to_word[i])
      v = model.embedding( torch.tensor(word_to_ix[word], dtype=torch.long ))
      print('size = {}, values = {}'.format(v.size(),v))
      #f.write('{}\t'.format(ix_to_word[i]))
      for value in v:  
        f.write('{}\t'.format(value))
      f.write('\n')

      # if i == 10:
      #   break;
      #Vector = [  ]

with open('/content/drive/My Drive/DSi/DATA_AND_SAVE_MODEL/bangla_word_embedding_CBOW_meta1.tsv', 'w') as f:
  with torch.no_grad():
    for i, word in enumerate(word_to_ix):
      #print(i, word)
      #print(ix_to_word[i])
      #v = model.embedding( torch.tensor(word_to_ix[word], dtype=torch.long ))
      #print('size = {}, values = {}'.format(v.size(),v))
      #f.write('{}\t'.format(ix_to_word[i]))  
      f.write('{}\n'.format(ix_to_word[i]))

Streaming output truncated to the last 5000 lines.
        -2.8026e-45,  0.0000e+00,  1.4013e-45, -2.8026e-45,  2.8026e-45,
        -1.4013e-45, -2.8026e-45, -1.4013e-45,  1.4013e-45,  4.2039e-45,
        -2.8026e-45, -1.4013e-45, -1.4013e-45, -0.0000e+00, -1.4013e-45,
        -0.0000e+00,  2.8026e-45,  0.0000e+00, -0.0000e+00,  2.8026e-45,
        -0.0000e+00,  1.4013e-45,  1.4013e-45,  0.0000e+00, -1.4013e-45,
         1.4013e-45,  0.0000e+00,  1.4013e-45, -1.4013e-45,  1.4013e-45,
         1.4013e-45,  1.4013e-45,  1.4013e-45,  0.0000e+00])
size = torch.Size([64]), values = tensor([ 1.4013e-45, -0.0000e+00, -1.4013e-45, -2.8026e-45, -0.0000e+00,
        -0.0000e+00, -1.4013e-45, -2.8026e-45,  0.0000e+00, -2.8026e-45,
        -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,  1.4013e-45,
         1.4013e-45,  1.4013e-45,  2.8026e-45,  2.8026e-45, -2.8026e-45,
         0.0000e+00,  1.4013e-45, -1.4013e-45, -1.4013e-45, -1.4013e-45,
        -1.4013e-45,  1.4013e-45,  0.0000e+00, -4.2